# DSI Explorer Agent

The role of this agent is to allow you to interface with Data Catalogue provided through the Data Science Infrasture (DSI) project.

To use this agent, you will need:
 - Access to Chat-GPT. The LANL AI team should be able to grant you access to this. The current model in use is "chatgpt-5.1"

Some capabilities available to the agent, besides the ability to search databases, are:
 - Download datasets
 - Email generation
 - web search
 - arxiv paper search
 - wikipedia search

These are all demonstrated in the notebook below.  

**Note:**
- LLMs can make mistakes and can exhibit random behavior from time to time.

## System setup  
Do not change!!!

In [ ]:
from pathlib import Path
from ursa.agents import DSIAgent
from langchain_openai import ChatOpenAI
from IPython.display import display, Markdown

In [ ]:
# Function to display and image in Jupyter notebook
from PIL import Image
def display_image(img_path: str):
    img = Image.open(img_path)
    display(img)

In [ ]:
import os
os.environ["SSL_CERT_FILE"] = "/home/pascalgrosset/certificates/Zscaler-RootCA-2041.crt"
os.environ["REQUESTS_CA_BUNDLE"] = "/home/pascalgrosset/certificates/Zscaler-RootCA-2041.crt"
os.environ["CURL_CA_BUNDLE"] = "/home/pascalgrosset/certificates/Zscaler-RootCA-2041.crt"


## Load Dataset
Please provide the agent with the location of the master dataset (containing datacards)

In [ ]:
#dataset_path = input("Please enter the address of the dataset to explore:")  # when in production
dataset_path = "data/oceans_11/ocean_11_datasets.db"

In [ ]:
# Specify the model to use
model = ChatOpenAI( model="gpt-5.1", max_tokens=100000, timeout=None, max_retries=2)

In [ ]:
# Initialize the agent
ai = DSIAgent(llm=model, db_index_name=dataset_path)

### Query the datasets

Use ```ai.ask("<query>")``` to query the dataset.  
  - e.g. ```ai.ask("Tell me about the datasets you have.")```

You should be able to query each of the datasets and ask the agent to load them or load back the master diana database.

In [ ]:
ai.ask("Tell me about the datasets you have.")

In [ ]:
ai.ask("Do you have any datasets loaded?")

In [ ]:
ai.ask("who are you?")

In [ ]:
ai.ask("what datasets do you have?")

In [ ]:
ai.ask("do you have access to any tools")

## Diving into the Computation of National Ignition Facility Shot dataset

In [ ]:
ai.ask("Do you have any implosion data?")

In [ ]:
ai.ask("Tell me everything you have about that dataset")

In [ ]:
ai.ask("Can you load that DSI database")

In [ ]:
ai.ask("list the tables/variables")

In [ ]:
ai.ask("show me some rows from nif_metadata")

In [ ]:
ai.ask("Can you download the data for sim name: nifcylxyz_hdf5_plt_cnt_0018")

In [ ]:
ai.ask("can you to visualize the data you downloaded with yt?") #you will need to install yt in yout environment for this

In [ ]:
display_image("nifcylxyz_hdf5_plt_cnt_0018_density_slice_z0.png")

In [ ]:
ai.ask("can you create a plot of dens_max over time")

In [ ]:
display_image("dens_max_vs_timestep.png")

In [ ]:
ai.ask("explain to me how you generated this plot")

In [ ]:
ai.ask("who is the owner of this dataset?")

In [ ]:
ai.ask("Can you email him and ask for more data. My name is Pascal and my email is pascalgrosset@lanl.gov")

## General Inquiries

In [ ]:
ai.ask("can you reload the master database now")

In [ ]:
ai.ask("Tell me again what datasets you have?")

In [ ]:
ai.ask("what is this Gray-Scott reaction-diffusion dataset?")

In [ ]:
ai.ask("can you find some arxiv papers related to this")

In [ ]:
ai.ask("Create a dashboard of the data you have")

In [ ]:
ai.ask("i need it in the browser and static html")

In [ ]:
ai.ask("I'm interested in asteroid impacts, is there anything related to it?")

In [ ]:
ai.ask("do asteroids really impact earth")

In [ ]:
ai.ask("can you search wikipedia for this impact: Chelyabinsk, Russia")